<a href="https://colab.research.google.com/github/alanLingZHANG/GPTAIAssistant/blob/main/Run_mistral_on_a_single_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runing Mistral-7b AI on a Single GPU with Google Colab
Welcome to this notebook that will show you how to load and run Mistral-7b with QLoRA which is a 4bit quantization technique with no performance degradation.

In this notebook, we will learn together how to load a model in 4bit, understand all its variants and how to run them for inference.

Note that this could be used for any model that supports device_map (i.e. loading the model with accelerate).

## Step 0 -  Enable text wrapping so we don't have to scrool horizontally


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)


## Step 1 - Install necessary packages
First, install the dependencies below to get started. As these features are available on the main branches only, we need to install the libraries below from source.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Step 2 - Define quantization parameters through the BitsandBytesConfig from transformers


* load_in_4bit=True: specify that we want to convert and load the model in 4-bit precision.
* bnb_4bit_use_double_quant=True: Use nested quantization for more memory efficient inference and training.
* bnd_4bit_quant_type="nf4": The 4bit integration comes with 2 different quantization types FP4 and NF4. The NF4 dtype stands for Normal Float 4 and is introduced in the QLoRA paper. By default, the FP4 quantization is used.
* bnb_4bit_compute_dype=torch.bfloat16: The compute dtype is used to change the dtype that will be used during computation. By default, the compute dtype is set to float32 but computation can be set to bf16 for speedups.



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



## Step 3 - Load the Model with quantization

Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

If you print the model, you will see that most of the nn.Linear layers are replaced by bnb.nn.Linear4bit layers!

In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

Let's make sure we loaded the whole model on GPU

In [ ]:
model.hf_device_map

{'': 0}

## Step 5 - Once loaded, run a generation!

Test 1: ask a recipe on mayonnaise

In [ ]:
device = "cuda:0"

messages = [
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]


encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)


generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] Do you have mayonnaise recipes? [/INST] Yes, here's a simple recipe for mayonnaise:

Ingredients:

* 3/4 cup vegetable oil
* 3/4 cup white wine vinegar
* 1 egg yolk
* 1 tablespoon finely chopped fresh herbs (optional)
* 1 teaspoon dijon mustard (optional)

Instructions:

1. In a large bowl, whisk together egg yolk, vinegar, mustard, and herbs (if using).
2. Slowly pour in the oil while whisking the mixture until it becomes creamy and smooth.
3. Refrigerate the mayonnaise for at least 1 hour to allow the flavors to develop.
4. Serve chilled and store in an airtight container in the refrigerator for up to a week.

Note: If you want to make dairy-free mayonnaise, you can substitute the oil with coconut oil or oil made from nuts like walnuts or pecans.</s>


Test 2: ask the model to generate python code from natural language instruction

In [ ]:
messages = [
    {"role": "user", "content": "write a python function to generate a list of random 1000 numbers between 1 and 10000?"}
    ]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] write a python function to generate a list of random 1000 numbers between 1 and 10000? [/INST] Sure, here is a simple Python function that generates a list of 1000 random numbers between 1 and 10000:
```
import random

def generate_random_list():
    # generate an empty list
    random_list = []

    # generate 1000 random numbers between 1 and 10000
    for i in range(1, 10001):
        # append the random number to the list
        random_list.append(random.randint(1, 10000))

    # return the list
    return random_list

# call the function
print(generate_random_list())
```
The `random.randint` function is used to generate a random integer between the two arguments provided. In this case, it is used to generate a random number between 1 and 10000, and the `range` function is used to loop over the numbers from 1 to 10000 to generate a list of 1000 random numbers.</s>


Test 3 : ask the model to explain in simple words what is a Large Language Model

In [ ]:
PROMPT= """ ### Instruction: Act as a data science expert.
### Question:
Explain to me what is Large Language Model. Assume that I am a 5-year-old child.

### Answer:
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])



<s>  ### Instruction: Act as a data science expert.

### Question:
Explain to me what is Large Language Model. Assume that I am a 5-year-old child.

 ### Answer:
 
A large language model is like a big brain that can understand and talk about many things. It uses a lot of computer power to learn and remember how people talk and write. Just like how you learn new things in school, the language model learns from a huge amount of information about the world. And because it's so smart, it can help people find information or write stories and poems, just by asking it questions!</s>


In [ ]:
device = "cuda:0"
PROMPT= """
### Question:
我想去中国旅游。 你推荐参观哪些地方

### Answer:
"""

encodeds = tokenizer(PROMPT, return_tensors="pt", add_special_tokens=True)
model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

<s>  
### Question:
我想去中国旅游。 你推荐参观哪些地方

### Answer:
中国是一个非常大国家，有许多非常漂亮的地方，因此我非常高兴能对你推荐一些参观方案。首先，你可以去中国的首都北京，看看中国历史的首要地标，像皇帝宫殿、辛阳寺、和祥楼之类的。

第二，这里也有许多其他的好地方，例如长城、西天门等。这里我们还可以参观一些非常著名的国家地标，如太阳山、大理城市、香港、鸡城、清原草原等.

第三，如果你喜欢 морpherous 地方，我们还可以去参观中国的海洋地marks，像海洋大象之地、海洋美景之地、海洋世界之地这些地方都非常漂亮。如果你喜欢中国的草原地带，我们可以参观青海草原、青海杜克、青海新革、青海海西、青海海东等地位。

最后，我还想推荐一个非常美景之地--太阳山，它是中国历史、文化和自然的一个非常重要的地标。这里有许多文化的风格，例如古老的住宅、古老的寺庙、古老的草坪，以及许多自然的风格，如美景的湖泊、美景的森林、美景的山区和美景的水falls。

总的来说，去中国旅游有许许多漂亮的地方。我希望我能为您提供一个帮助好的建议，以便您能根据自己的喜好和喜好来参观中国的好地方。</s>
